In [1]:
import pandas as pd
import collections

In [2]:
locs = pd.read_csv("ais_3/locs.csv")
locs["iso2"] = locs["name"].str.replace("^.+\(([^)]+)\)$", lambda g: g.group(1), regex=True)
locs

,name,longitude,latitude,iso2
0,SHAKOTAN (RU),146.833333,43.866667,RU
1,EL GHARDAQA (EG),33.850000,27.216667,EG
2,RAS SHUKHIER (EG),33.283333,28.133333,EG
3,RAS GHARIB (EG),33.100000,28.350000,EG
4,AS SUWAYS (EG),32.550000,29.966667,EG
...,...,...,...,...
167,PORT BERINGOVSKY (RU),179.366667,63.066667,RU
168,SALIF (YE),42.666667,15.300000,YE
169,RENI (UA),28.300000,45.433333,UA
170,Suez (EG),32.355877,30.443370,EG


In [3]:
df = pd.read_csv("all_ships.csv")
df

,vessel_type,mmsi,date_year,date_month,geo_name,longitude,latitude,iso2
0,Tanker,S219598000,2018,12,AIN SUKHNA TERMINAL (EG),32.366667,29.583333,EG
1,Tanker,S235108533,2018,12,AIN SUKHNA TERMINAL (EG),32.366667,29.583333,EG
2,Cargo,S248206000,2018,12,AIN SUKHNA TERMINAL (EG),32.366667,29.583333,EG
3,Tanker,S248642000,2018,12,AIN SUKHNA TERMINAL (EG),32.366667,29.583333,EG
4,Tanker,S249437000,2018,12,AIN SUKHNA TERMINAL (EG),32.366667,29.583333,EG
...,...,...,...,...,...,...,...,...
558098,Cargo,S565762000,2023,10,ZEIT BAY LPG TERMINAL (EG),33.566667,27.800000,EG
558099,Tanker,S565829000,2023,10,ZEIT BAY LPG TERMINAL (EG),33.566667,27.800000,EG
558100,Tanker,S622120925,2023,10,ZEIT BAY LPG TERMINAL (EG),33.566667,27.800000,EG
558101,Tanker,S622120926,2023,10,ZEIT BAY LPG TERMINAL (EG),33.566667,27.800000,EG


In [4]:
list(df.columns)

['vessel_type',
 'mmsi',
 'date_year',
 'date_month',
 'geo_name',
 'longitude',
 'latitude',
 'iso2']

In [5]:
df[df["iso2"] == "UA"]

,vessel_type,mmsi,date_year,date_month,geo_name,longitude,latitude,iso2
398,Cargo,S273334900,2018,12,ALUSHTA (UA),34.416667,44.683333,UA
399,Cargo,S273339300,2018,12,ALUSHTA (UA),34.416667,44.683333,UA
400,Cargo,S273430620,2018,12,ALUSHTA (UA),34.416667,44.683333,UA
401,Cargo,S312368000,2018,12,ALUSHTA (UA),34.416667,44.683333,UA
402,Cargo,S457711000,2018,12,ALUSHTA (UA),34.416667,44.683333,UA
...,...,...,...,...,...,...,...,...
557892,Tanker,S677027900,2023,10,YEVPATORIYA (UA),33.383333,45.200000,UA
558058,Cargo,S319152800,2023,10,YUZHNYY (UA),31.016667,46.600000,UA
558059,Cargo,S636014812,2023,10,YUZHNYY (UA),31.016667,46.600000,UA
558060,Cargo,S636019682,2023,10,YUZHNYY (UA),31.016667,46.600000,UA


In [6]:
ship_bags = {}

def hnd(cur):
    mmsi = cur["mmsi"].iloc[0]
    places = cur.value_counts(["date_year", "date_month", "iso2", "geo_name"]).reset_index()
    places = places.sort_values(["date_year", "date_month"])
    prev_ports = set()
    cur_month = None
    cur_year = None
    cur_ports = set()
    if mmsi == "S667001896":
        print(places)
    for _, row in places.iterrows():
        if row["date_year"] != cur_year or row["date_month"] != cur_month:
            if cur_year is not None and cur_month is not None:
                ship_bags[(mmsi, cur_year, cur_month)] = cur_ports | prev_ports
            prev_ports = set()
            if (
                    cur_year is None or cur_month is None
                    or (row["date_year"] == cur_year and row["date_month"] == cur_month + 1)
                    or (row["date_year"] == cur_year + 1 and row["date_month"] == 1 and cur_month == 12)):
                prev_ports = cur_ports
            cur_month = row["date_month"]
            cur_year = row["date_year"]
            cur_ports = set()
        cur_ports.add(row["iso2"])
        if row["geo_name"] == "Suez (EG)":
            cur_ports.add("SUEZ_CANAL")
        if row["geo_name"] == "Bosphorus (TR)":
            cur_ports.add("BOSPHORUS_STRAIT")
    if cur_year is not None and cur_month is not None:
        ship_bags[(mmsi, cur_year, cur_month)] = cur_ports | prev_ports

df.groupby("mmsi").apply(hnd)

len(ship_bags)

     date_year  date_month iso2          geo_name  0
29        2018          12   EG   EL MAADIYA (EG)  1
0         2019           1   EG   EL MAADIYA (EG)  2
83        2019           1   UA      YUZHNYY (UA)  1
84        2019           1   UA    MYKOLAYIV (UA)  1
85        2019           1   TR    Bosphorus (TR)  1
..         ...         ...  ...               ... ..
34        2023           7   UA  YEVPATORIYA (UA)  1
35        2023           7   UA         RENI (UA)  1
32        2023           8   UA         RENI (UA)  1
33        2023           8   TR    Bosphorus (TR)  1
114       2023          10   TR    Bosphorus (TR)  1

[115 rows x 5 columns]


269492

In [7]:
ship_bags[("S667001896", 2022, 10)]

{'BOSPHORUS_STRAIT', 'EG', 'TR', 'UA'}

In [8]:
res = collections.defaultdict(lambda: {"total": 0, "UA": 0, "RU": 0, "SUEZ_CANAL": 0, "BOSPHORUS_STRAIT": 0})

nan_count = 0
for _, row in df.iterrows():
    if f"{row['mmsi']}" == "nan":
        nan_count += 1
        continue
    ship_key = (row["mmsi"], row["date_year"], row["date_month"])
    port_key = (row["geo_name"], row["vessel_type"], row["date_year"], row["date_month"])
    ports = ship_bags[ship_key]
    res[port_key]["total"] += 1
    if "UA" in ports:
        res[port_key]["UA"] += 1
    if "RU" in ports:
        res[port_key]["RU"] += 1
    if "SUEZ_CANAL" in ports:
        res[port_key]["SUEZ_CANAL"] += 1
    if "BOSPHORUS_STRAIT" in ports:
        res[port_key]["BOSPHORUS_STRAIT"] += 1
    
nan_count

345

In [9]:
agg_df = {
    "geo_name": [],
    "date_year": [],
    "date_month": [],
    "vessel_type": [],
    "total": [],
    "ships_from_ukraine": [],
    "ships_from_russia": [],
    "ships_through_suez": [],
    "ships_through_bosphorus": [],
}
for key, counts in res.items():
    agg_df["geo_name"].append(key[0])
    agg_df["vessel_type"].append(key[1])
    agg_df["date_year"].append(key[2])
    agg_df["date_month"].append(key[3])
    agg_df["total"].append(counts["total"])
    agg_df["ships_from_ukraine"].append(counts["UA"])
    agg_df["ships_from_russia"].append(counts["RU"])
    agg_df["ships_through_suez"].append(counts["SUEZ_CANAL"])
    agg_df["ships_through_bosphorus"].append(counts["BOSPHORUS_STRAIT"])

port_df = pd.DataFrame(agg_df).merge(locs, left_on="geo_name", right_on="name").sort_values(
    ["geo_name", "date_year", "date_month"])
port_df = port_df.drop(columns=["name"]).reset_index(drop=True)
port_df

,geo_name,date_year,date_month,vessel_type,total,ships_from_ukraine,ships_from_russia,ships_through_suez,ships_through_bosphorus,longitude,latitude,iso2
0,ADLER (RU),2019,4,Tanker,1,0,1,0,1,39.933333,43.433333,RU
1,ADLER (RU),2020,1,Tanker,1,0,1,0,0,39.933333,43.433333,RU
2,ADLER (RU),2020,1,Cargo,1,0,1,0,0,39.933333,43.433333,RU
3,ADLER (RU),2021,4,Cargo,1,1,1,0,1,39.933333,43.433333,RU
4,ADLER (RU),2021,11,Cargo,1,1,1,0,1,39.933333,43.433333,RU
...,...,...,...,...,...,...,...,...,...,...,...,...
7551,ZEIT BAY TERMINAL (EG),2022,3,Cargo,1,1,0,1,1,33.600000,27.833333,EG
7552,ZEIT BAY TERMINAL (EG),2022,7,Cargo,2,0,0,2,0,33.600000,27.833333,EG
7553,ZEIT BAY TERMINAL (EG),2022,9,Cargo,2,0,0,2,1,33.600000,27.833333,EG
7554,ZEIT BAY TERMINAL (EG),2022,9,Tanker,1,0,0,0,0,33.600000,27.833333,EG


In [10]:
port_df.to_csv("ports.csv", index=False)

In [11]:
port_df_final = port_df.copy()

In [12]:
# treat straits as own countries
port_df_final.loc[port_df_final["geo_name"] == "Suez (EG)", "iso2"] = "Suez Canal"
port_df_final.loc[port_df_final["geo_name"] == "Bosphorus (TR)", "iso2"] = "Bosphorus Strait"

In [13]:
port_df_final["date"] = pd.to_datetime(port_df_final['date_year'].astype(str) + "-" + port_df_final['date_month'].astype(str), format='%Y-%m')

In [14]:
port_df_final = port_df_final[port_df_final["date"] > port_df_final["date"].min()]  # the first month won't have correct info on UA,RU,SUEZ,BOSPHORUS

In [15]:
port_df_final = port_df_final.sort_values(["date", "iso2", "vessel_type"])

In [16]:
port_df_final

,geo_name,date_year,date_month,vessel_type,total,ships_from_ukraine,ships_from_russia,ships_through_suez,ships_through_bosphorus,longitude,latitude,iso2,date
1342,Bosphorus (TR),2019,1,Cargo,1359,658,642,63,1359,29.071833,41.118333,Bosphorus Strait,2019-01-01
1341,Bosphorus (TR),2019,1,Tanker,443,74,293,33,443,29.071833,41.118333,Bosphorus Strait,2019-01-01
1891,DORALEH (DJ),2019,1,Cargo,98,1,3,16,4,43.083333,11.600000,DJ,2019-01-01
1892,DORALEH (DJ),2019,1,Tanker,30,0,0,2,0,43.083333,11.600000,DJ,2019-01-01
9,AIN SUKHNA TERMINAL (EG),2019,1,Cargo,4,0,1,0,2,32.366667,29.583333,EG,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5521,RENI (UA),2023,10,Tanker,53,53,1,1,28,28.300000,45.433333,UA,2023-10-01
5805,SEVASTOPOL (UA),2023,10,Tanker,1,1,0,0,1,33.533333,44.616667,UA,2023-10-01
6733,UST-DUNAISK (UA),2023,10,Tanker,22,22,0,0,16,29.700000,45.466667,UA,2023-10-01
6918,YALTA (UA),2023,10,Tanker,1,1,1,1,1,34.166667,44.483333,UA,2023-10-01


In [17]:
port_df_final.to_csv("ports_final.csv", index=False)